In [12]:
import pydst
import tkinter as tk
import pandas as pd

master = tk.Tk()

menu_items = {"eggs":7, "chips":1, "beer":2}
selected = {}

def calculate():
    bill = 0
    for item in menu_items:
        if selected[item].get():
            bill += menu_items[item]

    results.delete(1.0, tk.END)
    results.insert(tk.END, "Total cost - £" + str(bill))

for item in menu_items:
    is_selected = tk.BooleanVar()
    button = tk.Checkbutton(master, text=item, variable=is_selected)
    button.grid(sticky="w")
    selected[item] = is_selected

purchase_btn = tk.Button(master, text="Calculate", command=calculate)
purchase_btn.grid()

results = tk.Text(master)
results.grid(columnspan=2)

master.mainloop()